In [30]:
import mwxml
import re
from tqdm import tqdm

dump = mwxml.Dump.from_file(open("../data/dewiki-20220520-pages-articles-multistream1.xml")) #dewiki-20220520-pages-articles-multistream1.xml
print(dump.site_info.name, dump.site_info.dbname)

Wikipedia dewiki


In [ ]:
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))

#lines = []
with open("../data/title-categories-map.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        line += page.title + ","
        for revision in page:
            for match in pattern.finditer(revision.text):
                line += match.group(1) + ","
        line = line.strip(",")
        #lines.append(line)
        if words_re.search(str(line)):
            outfile.write(line+"\n")

In [ ]:
num_lines = sum(1 for line in open('data/title-categories-map.txt'))
print(num_lines)


In [109]:
#getting the infobox of a company

infobox = re.compile('((?<={{Infobox Unternehmen..).*?(?<=Homepage).*?(}}))')
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))


with open("../data/infobox.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ''
        for revision in page: 
            revision = re.sub(r'(\s?\n+\s+|\n)', '', revision.text)
            for match in infobox.finditer(revision):
                line += match.group(1) + "\n" 
                outfile.write(line+ '\n')
    

0it [00:00, ?it/s]


In [108]:
num_lines = sum(1 for line in open('../data/infobox.txt'))
print(num_lines)
#Output: 1400

611


In [135]:
#Companies without infobox
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
no_infobox = re.compile('(?!.*?Infobox)^.*$') #gets the whole text if it doesn´t contain 'Infobox'
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))

pattern_name  = re.compile('((?<=\''').*?(?=\'''))') #company name between ''' '''
pattern_website = re.compile('.*((?<=Weblinks...).*?(?=...Einzelnachweise)).*') 



with open("../data/company_name&website.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        website = ""
        line += page.title + ","
        for revision in page:
            revision = re.sub(r'(\s?\n+\s+|\n)', '', revision.text)
            for match in pattern.finditer(revision):
                line += match.group(1)  
        
        if words_re.search(str(line)) and no_infobox.finditer(revision):
            outfile.write(page.title + '\n')
            for match in pattern_website.finditer(revision):
                website += match.group(1)
                outfile.write(website + "\n")

#oft Matches mit der Kategorie Unternehmensart --> rausnehmen 

4038it [06:56,  9.71it/s] 


KeyboardInterrupt: 

In [ ]:
num_lines = sum(1 for line in open('../data/company_name&website.txt'))
print(num_lines)
#Output: 

In [36]:
pattern_firstsentence = re.compile('.*((?<=\''' ).*?(?<=\.))') 
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))

#Erster Satz fängt meistens mit dem Firmennamen in ''' ''' an. Dann wird der erste Punkt gesucht.

with open("../data/firstsentence.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        line += page.title
        firstsentence = ""
        for revision in page:
            #revision = re.sub(r'(\s?\n+\s+|\n)', '', revision.text)
            for match in pattern.finditer(revision.text):
                line += match.group(1)  
        
        if words_re.search(str(line)):
            outfile.write(page.title)
            for match in pattern_firstsentence.finditer(revision.text):
                firstsentence += match.group(1)
            outfile.write('\n'+ firstsentence+ '\n')



18143it [02:11, 137.62it/s] 


KeyboardInterrupt: 